# Proces Mining - fake dataset produceren

In [22]:
# Importeer packages
import pandas as pd
import numpy as np
import uuid
from random import normalvariate
import random
import datetime 
import weakref
from faker import Faker

## Classes en functies

In [ ]:
# Maakt een random datetime aan de hand van een startdatetime en een interval
def random_date(start,interval):
    if interval == 0:
        return start
    else:
        delta = (np.random.poisson(1,1)+1)+normalvariate(interval, interval*0.1)
        current = start + datetime.timedelta(minutes = delta[0])
        return current

In [23]:
# Elke stap is een class, met:
# - Een naam
# - Mogelijke volgende stappen (in lijst)
# - Kansen op die volgende stappen (in lijst)
# - Tijd binnen huidige stap (om verschil tussen start en eind te bepalen)
# - Tijd tussen huidige stap en volgende stap (verschil tussen eind huidige stap en begin volgende, in lijst)
# - Tijd waarbinnen de start en eindtijden moeten vallen, voor nu altijd tussen 8.00-18.00

class step():
    instances = []
    def __init__(self,name, next_step, chance_next_step, time_step,time_to_next_step, possible_interval =[8,18]):
        self.__class__.instances.append(weakref.proxy(self))
        self.name = name
        self.next_step = next_step
        self.chance_next_step = chance_next_step
        self.time_step = time_step
        self.time_to_next_step = time_to_next_step
        self.possible_interval = possible_interval

In [25]:
# Elk datapunt (een case van begin tot eind in het proces) is een class. 
class datapoint():
    # Starteigenschappen
    def __init__(self): # Begin met eerste stap van datapunt
        self.userID = uuid.uuid4()
        self.currentstep = Inschrijving
        
        # Startdatum random tussen 1-7-2018 en 1-10-2018, als niet in timewindow of in weekend, voeg telkens een uur toe
        fake = Faker()
        self.start = fake.date_time_between_dates(datetime_start=datetime.datetime(2018,7,1), datetime_end=datetime.datetime(2018,10,1))
        while self.start.hour < self.currentstep.possible_interval[0] or self.start.hour >= self.currentstep.possible_interval[1] or self.start.weekday()>=5:
            self.start = random_date(self.start,60) 
            
        # Einddatum. Als niet in timewindow of in weekend, voeg telkens een uur toe.
        self.end = random_date(self.start, self.currentstep.time_step)
        while self.end.hour < self.currentstep.possible_interval[0] or self.end.hour >= self.currentstep.possible_interval[1] or self.end.weekday()>=5:
            self.end = random_date(self.end,60) 
            
    # Functie die de case van begin tot eind in proces maakt    
    def create_datapoint(self,df):
        # Append eerste rij
        df = df.append({'ID':self.userID, 'Step':self.currentstep.name, 'Start':self.start, 'End':self.end}, ignore_index = True)     
        
        # Herhaal totdat de stap 'DONE' is bereikt
        while self.currentstep.name != 'DONE':
            rand = random.random()
            sumchance = 0
            
            # Vergelijk kansen met random nummer om volgende stap te bepalen
            for i in range(len(self.currentstep.chance_next_step)):
                sumchance += self.currentstep.chance_next_step[i]
                if rand < sumchance:
                    
                    # Loop through isntances to find correct one
                    for instance in step.instances:
                        if instance.name == self.currentstep.next_step[i]:
                            
                            # Update alle variabelen van nieuwe stap
                            self.start = random_date(self.end, self.currentstep.time_to_next_step[i])
                            while self.start.hour < self.currentstep.possible_interval[0] or self.start.hour >= self.currentstep.possible_interval[1] or self.start.weekday()>=5:
                                self.start = random_date(self.start,60)
                            self.currentstep = instance 
                            self.end = random_date(self.start, self.currentstep.time_step)
                            while self.end.hour < self.currentstep.possible_interval[0] or self.end.hour >= self.currentstep.possible_interval[1] or self.end.weekday()>=5:
                                self.end = random_date(self.end,60)
                                
                            # Als niet DONE, dan voeg toe aan df
                            if self.currentstep.name != 'DONE':
                                df = df.append({'ID':self.userID, 'Step':self.currentstep.name, 'Start':self.start, 'End':self.end}, ignore_index = True)
                            
                            break
                    break

        return df

In [26]:
# Functie die de hele dataset, tot aan AantalN maakt
def create_dataset(AantalN):
    df = pd.DataFrame(data={'ID':[], 'Step':[], 'Start':[], 'End':[]})
    while len(df) < AantalN:
        df = datapoint().create_datapoint(df)
    return df 

## Steps en dataset

In [24]:
# Het aanmaken van de verschillende stappen
# Instantiate new step : name, next steps, chances next steps, time during step, time to next step, possible times (optional)
week_min = 7*24*60
Inschrijving = step('Inschrijving', ['Registratie door arts','Wachten op patiënt info'],[0.9,0.1], 5, [120,0])
Registratie = step('Registratie door arts', ['Inschrijving', 'Controle door ZA'],[0.1,0.9], 10, [0,240])
Wachten = step('Wachten op patiënt info',['Inschrijving'], [1], 2*week_min,[0])
Controle = step('Controle door ZA',['Inschrijving','Registratie door arts','Controle door ZA','DONE','Facturatie'],[0.1,0.1,0.25,0.05,0.5],week_min, [0,0,0,0,240])
Facturatie = step('Facturatie',['Facturatie','Afronding','DONE','Controle door ZA'],[0.1,0.65, 0.05, 0.2], week_min,[0,week_min,0,0])
Afronding = step('Afronding',['Facturatie','DONE'],[0.4,0.6], 0,[0,0])
DONE = step('DONE',[],[],0,[0])

In [27]:
# Het aanmaken van de dataset met lengte AantalN
dataset = create_dataset(10000)

In [28]:
# Sla op als csv
dataset.to_csv("Proces_Mining_dataset_v2.csv")